In [6]:
import pandas as pd
import numpy as np
import regex as re 

from openai import OpenAI
import requests
from tqdm import tqdm
import os
from dotenv import load_dotenv
load_dotenv()

In [19]:
df = pd.read_csv('news_data.csv', usecols=['date', 'title', 'description'])

In [ ]:
## to datetime

# df.loc[df.date.str.contains('\d{2}/\d{2}/\d{4}', regex=True), 'date'] = df.date + ' - 00:00 AM'
# df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y -  %H:%M %p')

# Filter only related news by keywords

In [14]:
#  filter news containing VNI, VN30F1M
# keywords = ['vni', 'vn-index', 'vnindex', 'vn index', 'vn30f1m', 'vn30', 'phái sinh', 'đáo hạn', 'phai sinh', 'index', \
#             'dow jones', 'dji', 'hợp đồng tương lai', 'dj']
keywords = ['vn30f1m', 'vn30', 'phái sinh', 'đáo hạn', 'phai sinh', 'hợp đồng tương lai',\
            'dow jones', 'dji', 'dj']
pattern = '|'.join(map(re.escape, keywords)) 
cleaned = df[(df.title.str.lower().str.contains(pattern)) | (df.description.str.lower().str.contains(pattern))] 
# df_c[(df_c.title.str.lower().str.contains(pattern))]

In [9]:
cleaned

,date,title,description
24,02-06-2017 - 09:45 AM,Day trading - Chiến lược mua bán ngay trong ng...,Cơ hội để tìm kiếm lợi nhuận trong ngày trên t...
151,12-06-2017 - 09:00 AM,Các cách thức thao túng giá trên thị trường ch...,"Trên thế giới, các đối tượng thao túng giá thư..."
218,17-06-2017 - 09:58 AM,Chứng khoán phái sinh: Minh bạch thị trường th...,"Những ngày gần đây, khái niệm thị trường chứng..."
372,30-06-2017 - 08:43 AM,Thị trường phái sinh dự kiến khai trương đầu t...,Đại diện Sở Giao dịch Chứng khoán Hà Nội (HNX)...
491,10-07-2017 - 15:08 PM,Chứng khoán 6 tháng cuối năm: Tâm điểm có thuộ...,Thị trường chứng khoán (TTCK) Việt Nam đã trải...
...,...,...,...
4712,29-07-2018 - 21:13 PM,Thanh khoản thị trường phái sinh tiếp tục sụt ...,Số lượng vị thế mở ở mỗi phiên giao dịch đã tă...
4804,07-08-2018 - 08:45 AM,Công ty chứng khoán nhận định thị trường 7/8,Ngưỡng kháng cự trung hạn MA50 tại 961 điểm vẫ...
4806,07-08-2018 - 07:10 AM,"FTS, PTB, SBT, NLG, HBC, SCR, FTM, CEE, LTG, F...",Thông tin giao dịch lượng lớn cổ phiếu trên HN...
4818,08-08-2018 - 09:21 AM,"Dòng tiền ồ ạt chảy vào phái sinh, CTCK đua nh...","Theo xu hướng dòng tiền, các CTCK hiện đang đu..."


In [16]:
cleaned.to_csv('cleaned_data.csv', index=False)

# Label data

In [ ]:
together_api = os.getenv('together_api')

endpoint = 'https://api.together.xyz/inference'
headers={
    "Authorization": f"Bearer {together_api}",
    "User-Agent": "News Labeling"
}

In [ ]:
df = cleaned

In [ ]:
df['content'] = df['title'] + ' ' + df['description']
df['result'] = ''

In [ ]:
result = []
for i in tqdm(range(877, len(df))):
    text = df.iloc[i].content

    client = OpenAI(
    api_key=together_api,
    base_url='https://api.together.xyz/v1',
    )

    chat_completion = client.chat.completions.create(
    messages=[
        {
        "role": "system",
        "content": "Perform sentiment analysis about Vietnamese news content",
        },
        {
        'role': 'system',
        'content': 'Label the sentences, output format as either: "positive", "negative", "mixed", or "neutral. Only return one of these four\
            options. Do not explain more"',
        },
        {
        "role": "user",
        "content": text,
        }
    ],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1"
    )
    res = chat_completion.choices[0].message.content
    df.loc[i, 'result'] = res

# Extract Label

In [ ]:
def extract_text(text):
    if 'neutral' in text:
        return '1'
    if 'negative' in text:
        return '0'
    if 'mixed' in text:
        return '2'
    else:
        return '3'
    
df['result'] = df['result'].apply(extract_text)